In [ ]:
%load_ext sql

In [ ]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from faker import Faker

In [ ]:
pd.set_option('display.max_rows', 12)
pd.set_option('display.min_rows', 6)

In [ ]:
fake = Faker()

base_url = "http://fastapi:8000"
headers = {"Content-Type": "application/json"}

print("customers")
# Create customers
for customer_id in range(1, 11):
    data = {
        "first_name": fake.first_name(),
        "last_name": fake.last_name(),
        "email": fake.email(),
        "joined_at": fake.iso8601()
    }
    response = requests.post(f"{base_url}/customers/", json=data, headers=headers)
    print(response.status_code, response.json())

print("vendors")
# Create vendors
for vendor_id in range(1, 6):
    data = {
        "vendor_name": fake.company(),
        "region": fake.country_code()
    }
    response = requests.post(f"{base_url}/vendors/", json=data, headers=headers)
    print(response.status_code, response.json())

print("inventory per vendor")
# Add inventory per vendor
for vendor_id in range(1, 6):
    for item_id in range(1, 11):
        params = {"vendor_id": vendor_id}
        data = {
            "item_name": fake.word(),
            "category": fake.word(),
            "price": fake.random_int(min=10, max=500),
        }
        response = requests.post(f"{base_url}/inventory/", json=data, headers=headers, params=params)
        print(response.status_code, response.json())

print("cart and checkout twice and add to cart again!")
for customer_id in range(1, 11):
    for _ in range(2):  # Checkout twice
        cart_data = {
            "user_id": customer_id,
            "cart": [
                {
                    "item_id": str(fake.random_int(min=1, max=10)),
                    "vendor_id": str(fake.random_int(min=1, max=5)),
                    "qty": fake.random_int(min=1, max=5)
                }
            ]
        }
        response = requests.post(f"{base_url}/cart/", json=cart_data, headers=headers)
        print(response.status_code, response.json())
        
        response = requests.post(f"{base_url}/checkout/", json={"user_id": customer_id}, headers=headers)
        print(response.status_code, response.text)
    
    # Add to cart one more time
    cart_data = {
        "user_id": customer_id,
        "cart": [
            {
                "item_id": str(fake.random_int(min=1, max=10)),
                "vendor_id": str(fake.random_int(min=1, max=5)),
                "qty": fake.random_int(min=1, max=5)
            }
        ]
    }
    response = requests.post(f"{base_url}/cart/", json=cart_data, headers=headers)
    print(response.status_code, response.json())

In [ ]:
connection_string = f"postgresql://user:password@db:5432/ecommerce"
engine = create_engine(connection_string)
get_ipython().run_line_magic('sql', connection_string)

In [ ]:
%sql \l

In [ ]:
%sql \dt

In [ ]:
table_list = [
    "customer", "inventory", "vendor"
]

In [ ]:
for table in table_list:
    print(f"{table}")
    schema_query = f"columns --table {table}"
    display(get_ipython().run_line_magic('sqlcmd', schema_query))

In [ ]:
for table in table_list:
    print(f"{table}")
    display(pd.read_sql(f"select * from {table} limit 100", engine))